In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

c:\Users\jaesc2\GitHub\skforecast
0.18.0


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.plot.plot import backtesting_gif_creator

In [3]:
y = pd.Series(np.arange(100))
# y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 7,
        initial_train_size    = 70,
        window_size           = 5,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = True,
        gap                   = 5,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
folds = cv.split(X=y, as_pandas=False)
folds

Information of folds
--------------------
Number of observations used for initial training: 70
Number of observations used for backtesting: 30
    Number of folds: 4
    Number skipped folds: 0 
    Number of steps per fold: 7
    Number of steps to exclude between last observed data (last window) and predictions (gap): 5
    Last fold only includes 4 observations.

Fold: 0
    Training:   0 -- 69  (n=70)
    Validation: 75 -- 81  (n=7)
Fold: 1
    Training:   7 -- 76  (n=70)
    Validation: 82 -- 88  (n=7)
Fold: 2
    Training:   14 -- 83  (n=70)
    Validation: 89 -- 95  (n=7)
Fold: 3
    Training:   21 -- 90  (n=70)
    Validation: 96 -- 99  (n=4)



[[0, [0, 70], [65, 70], [70, 82], [75, 82], True],
 [1, [7, 77], [72, 77], [77, 89], [82, 89], True],
 [2, [14, 84], [79, 84], [84, 96], [89, 96], True],
 [3, [21, 91], [86, 91], [91, 100], [96, 100], True]]

In [4]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = OneStepAheadFold(
        initial_train_size    = 70,
        window_size           = 3,
        differentiation       = None,
        return_all_indexes    = False,
        verbose               = False
    )
folds = cv.split(X=y)
folds

[[0, 70], [70, 100], True]

In [5]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.preprocessing import RollingFeatures
import re
import pytest
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from skforecast.sarimax import Sarimax
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterSarimax
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection import backtesting_sarimax

# Fixtures
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

from skforecast.recursive.tests.tests_forecaster_sarimax.fixtures_forecaster_sarimax import y_datetime
from skforecast.recursive.tests.tests_forecaster_sarimax.fixtures_forecaster_sarimax import exog_datetime

from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import (
    series_wide_range,
    series_wide_dt,
    series_wide_dt_nans,
    series_long_dt,
    series_long_dt_nans,
    series_dict_range,
    series_dict_dt,
    series_dict_nans,
    exog_dict_nans,
    series_dict_nans_train,
    exog_dict_nans_train,
    custom_metric
)

In [16]:

forecaster = ForecasterSarimax(
                    regressor = Sarimax(order=(3, 2, 0), maxiter=1000, method='cg', disp=False)
                )

cv = TimeSeriesFold(
            steps                 = 2,
            initial_train_size    = len(y_datetime) - 12,
            fold_stride           = 4,
            refit                 = True,
            fixed_train_size      = True,
            gap                   = 0,
            allow_incomplete_fold = True
        )

metric, backtest_predictions = backtesting_sarimax(
                                    forecaster = forecaster,
                                    y          = y_datetime,
                                    cv         = cv,
                                    exog       = exog_datetime,
                                    metric     = 'mean_absolute_error',
                                    alpha      = None,
                                    interval   = None,
                                    verbose    = False
                                )

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# More decimals numpy
np.set_printoptions(precision=15, suppress=True)
metric.to_numpy()

array([[0.080745243332592]])

In [18]:
backtest_predictions

,fold,pred
2038-12-31,0,0.594091
2039-12-31,0,0.783234
2042-12-31,1,0.768954
2043-12-31,1,0.815940
2046-12-31,2,0.774603
2047-12-31,2,0.583977


In [19]:

# backtest_predictions = (
#     backtest_predictions
#     .rename_axis('idx', axis=0)
#     .set_index('level', append=True)
# )

(
    backtest_predictions
    .loc[~backtest_predictions.index.duplicated(keep='last')]
).head(30)

,fold,pred
2038-12-31,0,0.594091
2039-12-31,0,0.783234
2042-12-31,1,0.768954
2043-12-31,1,0.815940
2046-12-31,2,0.774603
2047-12-31,2,0.583977


In [20]:
np.set_printoptions(precision=8, suppress=True)
backtest_predictions[['fold', 'pred']].to_numpy()

array([[0.        , 0.59409098],
       [0.        , 0.78323365],
       [1.        , 0.76895405],
       [1.        , 0.81593985],
       [2.        , 0.77460313],
       [2.        , 0.5839772 ]])

In [21]:
backtest_predictions.index

DatetimeIndex(['2038-12-31', '2039-12-31', '2042-12-31', '2043-12-31',
               '2046-12-31', '2047-12-31'],
              dtype='datetime64[ns]', freq=None)

In [16]:
pd.DatetimeIndex(['2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06',
               '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08',
               '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12',
               '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14',
               '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16',
               '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18'])

DatetimeIndex(['2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06',
               '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08',
               '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12',
               '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14',
               '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16',
               '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18'],
              dtype='datetime64[ns]', freq=None)